In [1]:
import pandas as pd
import json

In [71]:
mpx_erc20_address = "0x66eEd5FF1701E6ed8470DC391F05e27B1d0657eb"
mlp_erc20_address = "0xd5c313DE2d33bf36014e6c659F13acE112B80a8E"
vault_address = "0x3CB54f0eB62C371065D739A34a775CC16f46563e"
mlp_manager_address = "0xA3Ea99f8aE06bA0d9A6Cf7618d06AEa4564340E9"
musd_erc20_address = "0xB7209EbCBF71c0ffA1585B4468A11CFfdcDBB9a9"
esmpx_erc20_address = "0xe0f606e6730bE531EeAf42348dE43C2feeD43505"
reward_router_address = "0x20De7f8283D377fA84575A26c9D484Ee40f55877"
reward_reader_address = "0x512F8D4E28EB53A6d036aEDA9C5a4D1De6DBD543"
weth_erc20_address = "0x21be370D5312f44cB42ce377BC9b8a0cEF1A4C83"
tracker_staked_mlp_address = "0xa4157E273D88ff16B3d8Df68894e1fd809DbC007" # staked MPX
tracker_staked_mpx_address = "0x49A97680938B4F1f73816d1B70C3Ab801FAd124B" # fee + staked MLP

## Load Fantom transactions

The first stage of this snapshot used an archive of the Fantom DAG, which was queried from public RPC servers.
The Fantom archive contains blocks ranging from 54647659 to 65653613 (2023-01-29 to 2023-07-12).

In [72]:
fantom_txs_filename = "../data/fantom-txs.csv.gz"
txs = pd.read_csv(fantom_txs_filename)
txs.head()

,block_id,tx_hash,method,from,to,quantity
0,54647659,0x68b471267b020fb7be707817bbe822da9a18f45071c7...,0x627dd56a,0x5cD5AFF4A3b42b59651BAB5Eb45c07375f60e905,0x3319161b131401124E61ffCDFf0aa7f6FDcbbFEd,0
1,54647659,0xa873edd8d865c95afd33d5ebeb34cad3df4843b2c769...,0x16711fdc,0x53e0466D46306e9824590988f50B0EEAcA1A6Dae,0x6Ff283ac7BB7F66475Ca79aA549c6f689cDB89Dd,0
2,54647659,0x68f2f5c608c220d3ff0c421eed9812b7e9d6124d4adb...,0xac882d06,0x656DAA21A427a1A290dD78AC581fFe8CbfF16066,0x6b120e310A892D1Cbe0F89E76Bf7Cb0da3a85BCe,0
3,54647659,0xd1e53d2e2bcf779b4061a4dc7080a33d4f270b72c61a...,0x627dd56a,0x01883C8E7Abb7C2Ef03d802F77AA567a254e17d7,0x0000900e00070d8090169000D2B090B67f0c1050,0
4,54647659,0x961463fa4ffc36d9d1fea04060b9c57072950a81a5ca...,0x58b58d44,0x211cB8e0740AdF9066453e627dcc196225c8F0E7,0xe0df6a0840C4f31E81C3d4B11Ba8B579F2ba8b1a,0


## Identify interacting wallet addresses

This snapshot begins by identifying all the addresses that ever interacted with Morphex contracts.
This list of addresses will be used in the next stage to query balances on-chain.

In [73]:
mpx_txs = txs.query(f'to == "{mpx_erc20_address}"')
len(mpx_txs)

10879

In [74]:
unique_mpx_addresses = set(mpx_txs['from'])
len(unique_mpx_addresses)

4233

In [75]:
reward_router_txs = txs.query(f'to == "{reward_router_address}"')
len(reward_router_txs)

46739

In [76]:
unique_reward_addresses = set(reward_router_txs['from'])
len(unique_reward_addresses)

1585

In [77]:
addresses = unique_reward_addresses | unique_mpx_addresses
with open("../data/fantom-addresses.json", "w") as f:
    json.dump(list(addresses), f)
len(addresses)

4501

## Call contracts on-chain to obtain balances

Fantom RPCs providing full/archival operation permit `eth_call` to be executed against a historical block.
For this snapshot, we queried contracts on-chain at block `64884840` (June 30, 2:20 UTC).

The script for querying the contracts is located in `/src/scripts/mlp.py`.

In [ ]:
def parse_address(item):
    return item.split(",")[2].split("'")[1]

### Staked MLP

In [88]:
with open("../data/fantom-staked-mlp-results.json", "r") as f:
    mlp_call_results = json.load(f)
len(mlp_call_results)

4501

In [89]:
mlp_balances = {}
for key, value in mlp_call_results.items():
    if value > 0:
        mlp_balances[parse_address(key)] = value

with open("../data/fantom-fsmlp-balances.json", "w") as f:
    json.dump(mlp_balances, f, indent=2)

sum(mlp_balances.values()) / 1e18

418265.70937368023

### Single-staked MPX

In [90]:
with open("../data/fantom-staked-mpx-results.json", "r") as f:
    mpx_call_results = json.load(f)
len(mpx_call_results)

4501

In [91]:
mpx_balances = {}
for key, value in mpx_call_results.items():
    if value > 0:
        mpx_balances[parse_address(key)] = value

with open("../data/fantom-smpx-balances.json", "w") as f:
    json.dump(mpx_balances, f, indent=2)

sum(mpx_balances.values()) / 1e18

7369371.241099392